In [34]:
# Flow_from_directory
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications.resnet import ResNet152
from tensorflow.keras.regularizers import L2


In [35]:
IMAGE_SIZE = 224

# Train data with validation split with data augmentation
train_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2,
                                     width_shift_range=0.2, height_shift_range=0.2,
                                     horizontal_flip=True, vertical_flip=True,
                                     brightness_range=[0.4,1.5],
                                     )
train= train_generator.flow_from_directory('data/full/calc/train/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=16, class_mode='binary', subset='training')
val= train_generator.flow_from_directory('data/full/calc/train/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=16, class_mode='binary', subset='validation')

# Test data
test_generator = ImageDataGenerator(rescale=1./255)
test = test_generator.flow_from_directory('data/full/calc/test/', target_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=16, class_mode='binary')

Found 988 images belonging to 2 classes.
Found 246 images belonging to 2 classes.
Found 286 images belonging to 2 classes.


In [36]:
pretrained_model = ResNet152(include_top=False,
                             pooling='max',
                             classes=2,
                             weights='imagenet'
                             )

model_layers = [
    pretrained_model,
    Dense(units=512, activation='relu', kernel_regularizer=L2(0.0001), bias_regularizer=L2(0.001)),
    Dropout(0.5),
    Dense(units=128, activation='relu', kernel_regularizer=L2(0.0001), bias_regularizer=L2(0.0001)),
    Dense(units=1, activation='sigmoid'),
]

model = Sequential(model_layers)

In [37]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train, validation_data=val, epochs=30)

In [ ]:
# Plot the training and validation accuracy and loss at each epoch
import matplotlib.pyplot as plt
acc = model.history.history['accuracy']
val_acc = model.history.history['val_accuracy']
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

In [ ]:
# Create roc curve
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import numpy as np

# Get the predictions from the model using the generator
predictions = model.predict(test)

# Get the true values from the generator
true_values = test.classes

# Get the predicted probabilities for each class
pred_probabilities = predictions

# Get the false positive rate and true positive rate
fpr, tpr, thresholds = roc_curve(true_values, pred_probabilities)

# Plot the ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='ResNet152')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')